# Importing Necesseties

In [1]:
!pip install -r ../resources/requirements.txt



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import re
import string
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
# Any results you write to the current directory are saved as output.

**Below is a helper Function which generates random colors which can be used to give different colors to your plots.Feel free to use it**

In [3]:
data = pd.read_csv('../../resources/data/Twitter_Emotion_Dataset.csv')
data

,label,tweet
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu..."
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi..."
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng..."
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata..."
...,...,...
4396,love,"Tahukah kamu, bahwa saat itu papa memejamkan m..."
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ..."
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...


t## Initial modal

In [4]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


data['tweet'] = data['tweet'].apply(lambda x: clean_text(x))
data

,label,tweet
0,anger,soal jln jatibarupolisi tdk bs gertak gubernur...
1,anger,sesama cewe lho kayaknya harusnya bisa lebih r...
2,happy,kepingin gudeg mbarek bu hj amad foto dari goo...
3,anger,jln jatibarubagian dari wilayah tn abangpengat...
4,happy,sharing pengalaman aja kemarin jam batalin ti...
...,...,...
4396,love,tahukah kamu bahwa saat itu papa memejamkan ma...
4397,fear,sulitnya menetapkan calon wapresnya jokowi di ...
4398,anger,masa depannya nggak jelas lha iya gimana mau ...
4399,happy,dulu beneran ada mahasiswa teknik ui nembak p...


In [5]:
indo_stop_words = stopwords.words(fileids='indonesian')
indo_stop_words.append("username")
indo_stop_words.append("url")
indo_stop_words.append("ya")
indo_stop_words.append("yg")


def remove_stopword(x):
    return " ".join([y for y in x.split() if y not in indo_stop_words])


data['tweet'] = data['tweet'].apply(lambda x: remove_stopword(x))

In [6]:
data

,label,tweet
0,anger,jln jatibarupolisi tdk bs gertak gubernur eman...
1,anger,cewe lho kayaknya rasain sibuk jaga rasain sak...
2,happy,kepingin gudeg mbarek bu hj amad foto google s...
3,anger,jln jatibarubagian wilayah tn abangpengaturan ...
4,happy,sharing pengalaman aja kemarin jam batalin tik...
...,...,...
4396,love,tahukah papa memejamkan matanya menahan gejola...
4397,fear,sulitnya menetapkan calon wapresnya jokowi pil...
4398,anger,depannya nggak lha iya gimana coba lulusan sen...
4399,happy,beneran mahasiswa teknik ui nembak pacarnya pa...


In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

# Encode labels
encode_labels = LabelEncoder().fit_transform(data["label"])
# Train-test bt_split

df = data

X = df["tweet"]
y = encode_labels

# 1. TF-IDF vectorizer (turns tweets into vectors)
tfidf = TfidfVectorizer(
    ngram_range=(1,2),       # unigrams + bigrams
    stop_words=None          # you can plug in Indonesian stopword list here
)

# 2. LSA via Truncated SVD
lsa = TruncatedSVD(n_components=100, random_state=42)

# 3. Classifier
clf = XGBClassifier(random_state=42)

# 4. Pipeline
model = Pipeline([
    ("tfidf", tfidf),
    ("lsa", lsa),
    ("clf", clf)
])

# Train/test bt_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Report
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.56      0.63      0.59       119
           1       0.69      0.56      0.62        62
           2       0.65      0.51      0.57       111
           3       0.67      0.87      0.76        55
           4       0.41      0.41      0.41        94

    accuracy                           0.58       441
   macro avg       0.59      0.60      0.59       441
weighted avg       0.58      0.58      0.57       441



## Trying basic NN

In [17]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

X_text = df["tweet"].values
y = encode_labels

# --------------------------
# TF-IDF + LSA
# --------------------------
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
lsa = TruncatedSVD(n_components=100, random_state=42)

X_tfidf = tfidf.fit_transform(X_text)
X_lsa = lsa.fit_transform(X_tfidf).astype(np.float32)   # (n_samples, 100)
X_tfidf, X_lsa

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 54663 stored elements and shape (4401, 5000)>,
 array([[ 0.02715662,  0.00668656, -0.0214614 , ...,  0.0580422 ,
          0.014818  , -0.06430656],
        [ 0.08823814, -0.01614558, -0.00832853, ...,  0.03948399,
         -0.04237257, -0.0142221 ],
        [ 0.05108306,  0.01388742, -0.02614813, ...,  0.05248163,
          0.02892105,  0.08285797],
        ...,
        [ 0.11985517, -0.02876377, -0.05878504, ..., -0.00741188,
         -0.04522653,  0.0497761 ],
        [ 0.08878586,  0.08303224, -0.00087154, ...,  0.00849323,
          0.03101327,  0.01118153],
        [ 0.08734218,  0.07794479, -0.03971541, ..., -0.04679065,
         -0.00307968, -0.06319682]], shape=(4401, 100), dtype=float32))

In [18]:
class TweetDataset(Dataset):
    def __init__(self, text_features, labels):
        self.text_features = torch.tensor(text_features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.text_features[idx], self.labels[idx]

# Train/test bt_split
X_train, X_test, y_train, y_test = train_test_split(X_lsa, y, test_size=0.2, random_state=42)

train_dataset = TweetDataset(X_train, y_train)
test_dataset = TweetDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2)
train_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x210cddeed10>,
 <torch.utils.data.dataloader.DataLoader at 0x210cde15c10>)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifierBiLSTM(nn.Module):
    def __init__(self, text_input_dim, hidden_dim, lstm_hidden_dim, output_dim, seq_len=10):
        super().__init__()
        self.seq_len = seq_len
        self.feature_dim = text_input_dim // seq_len  # bt_split LSA into timesteps

        # BiLSTM
        self.lstm = nn.LSTM(
            input_size=self.feature_dim,
            hidden_size=lstm_hidden_dim,
            num_layers=1,
            bidirectional=True,
            batch_first=True
        )

        # Fully connected layers
        self.fc1 = nn.Linear(lstm_hidden_dim * 2, hidden_dim)  # *2 for bidirectional
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, text_x):
        # text_x: (batch, text_input_dim)
        batch_size = text_x.size(0)

        # reshape into (batch, seq_len, feature_dim)
        x = text_x.view(batch_size, self.seq_len, self.feature_dim)

        # LSTM
        lstm_out, _ = self.lstm(x)   # (batch, seq_len, 2*lstm_hidden_dim)

        # take the last timestep output
        last_out = lstm_out[:, -1, :]   # (batch, 2*lstm_hidden_dim)

        # fully connected
        h = F.relu(self.fc1(last_out))
        out = self.fc2(h)
        return out


In [25]:

# --------------------------
# Training
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifierBiLSTM(
    text_input_dim=X_lsa.shape[1],  # must match LSA dims (e.g., 100)
    hidden_dim=64,
    lstm_hidden_dim=32,
    output_dim=len(set(y)),
    seq_len=10                      # e.g., 10 timesteps × 10 features = 100
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

# Train loop
for epoch in range(100):
    model.train()
    for text_x, labels in train_loader:
        text_x, labels = text_x.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(text_x)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 1.5681
Epoch 2, Loss: 1.5137
Epoch 3, Loss: 1.5206
Epoch 4, Loss: 1.6745
Epoch 5, Loss: 1.4555
Epoch 6, Loss: 1.8816
Epoch 7, Loss: 1.4900
Epoch 8, Loss: 1.4382
Epoch 9, Loss: 1.4840
Epoch 10, Loss: 1.4793
Epoch 11, Loss: 1.9024
Epoch 12, Loss: 1.4395
Epoch 13, Loss: 1.4382
Epoch 14, Loss: 1.4873
Epoch 15, Loss: 1.3965
Epoch 16, Loss: 1.6558
Epoch 17, Loss: 1.6983
Epoch 18, Loss: 1.4408
Epoch 19, Loss: 1.3997
Epoch 20, Loss: 1.6875
Epoch 21, Loss: 1.8952
Epoch 22, Loss: 1.4820
Epoch 23, Loss: 1.9063
Epoch 24, Loss: 1.6837
Epoch 25, Loss: 1.9204
Epoch 26, Loss: 1.4897
Epoch 27, Loss: 1.7014
Epoch 28, Loss: 1.4822
Epoch 29, Loss: 1.4396
Epoch 30, Loss: 1.3939
Epoch 31, Loss: 1.4844
Epoch 32, Loss: 1.6548
Epoch 33, Loss: 1.9036
Epoch 34, Loss: 1.9167
Epoch 35, Loss: 1.4404
Epoch 36, Loss: 1.6860
Epoch 37, Loss: 1.6984
Epoch 38, Loss: 1.6959
Epoch 39, Loss: 1.3964
Epoch 40, Loss: 1.4338
Epoch 41, Loss: 1.4831
Epoch 42, Loss: 1.4355
Epoch 43, Loss: 1.4813
Epoch 44, Loss: 1.68

In [24]:
# --------------------------
# Evaluation
# --------------------------
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for text_x, labels in test_loader:
        text_x, labels = text_x.to(device), labels.to(device)
        outputs = model(text_x)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print("Accuracy:", correct / total)


Accuracy: 0.5164585698070374
